# AMISC tutorial

## Simple 2-component system
\begin{align}
y_1 &= f_1(x) = x \sin(\pi x)\\
y_2 &= f_2(y_1) = \frac{1}{1 + 25y_1^2}
\end{align}

In [ ]:
from amisc import Variable, Component, System

In [ ]:
# Define component model functions
def f1(x):
    return {'y1': x['x'] * np.sin(np.pi * x['x'])}
def f2(y1):
    return {'y2': 1 / (1 + 25*y1['y1']**2)}

In [ ]:
# Define variables
x = Variable()
y1 = Variable()
y2 = Variable()

In [ ]:
# Link components together
comp1 = Component(f1, 'x', 'y1')
comp2 = Component(f2, 'y1', 'y2')
system = System(comp1, comp2)

In [ ]:
# Plot the true functions
import numpy as np
import matplotlib.pyplot as plt

x_grid = {'x': np.linspace(0, 1, 100)}
y1_grid = f1(x_grid)
y2_grid = f2(y1_grid)

fig, ax = plt.subplots(1, 2, figsize=(6, 3), layout='tight', sharey='row')
ax[0].plot(x_grid['x'], y1_grid['y1'])
ax[1].plot(x_grid['x'], y2_grid['y2'])
ax[0].set_xlabel('$x$')
ax[1].set_xlabel('$x$')
ax[0].set_ylabel('$f(x)$')
plt.show()

In [ ]:
# Can do the same thing with the system object
y_grid = system.predict(x_grid, use_model='best')

fig, ax = plt.subplots(1, 2, figsize=(6, 3), layout='tight', sharey='row')
ax[0].plot(x_grid['x'], y_grid['y1'], '-r')
ax[1].plot(x_grid['x'], y_grid['y2'], '-r')
ax[0].set_xlabel('$x$')
ax[1].set_xlabel('$x$')
ax[0].set_ylabel('$f(x)$')
plt.show()

### Add a new component model:
$$y_3(x, y_2) = x \cos(\pi y_2)$$

In [ ]:
# Add a new component
y3 = Variable()
f3 = lambda inputs: {'y3': inputs['x'] * np.cos(np.pi * inputs['y2'])}
comp3 = Component(f3, [x, y2], y3)
system.insert_component(comp3)
print(system)

In [ ]:
y_grid = system.predict(x_grid, use_model='best')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), layout='tight', sharey='row')
ax[0].plot(x_grid['x'], y_grid['y1'], '-r')
ax[1].plot(x_grid['x'], y_grid['y2'], '-r')
ax[2].plot(x_grid['x'], y_grid['y3'], '-r')
ax[0].set_xlabel('$x$')
ax[1].set_xlabel('$x$')
ax[2].set_xlabel('$x$')
ax[0].set_ylabel('$f(x)$')
plt.show()

### Constructing a surrogate for the multidisciplinary system

In [ ]:
# Construct and use a surrogate (literally these two lines right here)
system.fit()
y_surr = system.predict(x_grid)

## Model fidelities
Model fidelity is specified using tuples of integer indices (i.e. "multi-indices").

- $\alpha = (\alpha_1, \alpha_2, \dots)$ &rarr; controls the "physical" fidelity of the model    (_think_: mesh refinement, $\Delta t$, etc.)
- $\beta = (\beta_1, \beta_2, \dots)$ &rarr; controls the "parametric" fidelity of the surrogate (_think_: number of training points, etc.)

Construct a multi-fidelity surrogate for the model:

\begin{align}
y &= f(x) = \cos\left(\frac{\pi}{2}(x + \frac{4}{5} + \frac{\epsilon}{5})\right)\\
\epsilon &= 2^{-\alpha_0}\\
\text{for}\ \alpha_0 &= (0, 1, 2, \dots)\\
\end{align}

As the $\alpha$ index increases, $\epsilon\rightarrow 0$ and the "fidelity" of the model increases.

In [ ]:
def multilevel_model(inputs, alpha):
    eps = 2**(-alpha[0])
    return {'y': np.cos(np.pi/2 * (inputs['x'] + 4/5 + (1/5)*eps))}

x = Variable()
y = Variable()
comp = Component(multilevel_model, x, y, max_alpha=(2,), max_beta=(2,))

In [ ]:
x_grid = {'x': np.linspace(-1, 1, 100)}
y_truth = multilevel_model(x_grid, (10,))['y']
colors = ['r', 'g', 'b']

fig, ax = plt.subplots(figsize=(4, 3), layout='tight')
ax.plot(x_grid['x'], y_truth, '-k', label='Truth')
for i in range(3):
    y_grid = comp.predict(x_grid, use_model=(i,))
    ax.plot(x_grid['x'], y_grid['y'], f'-{colors[i]}', label=rf'$\alpha$={i}')
ax.legend()
plt.show()

## Random variables

In [ ]:
x = Variable(distribution='Normal(2, 0.5)')
y = Variable(distribution='Uniform(-1, 10)')
x_samples = x.sample(1000)
y_samples = y.sample(1000)

fig, ax = plt.subplots()
ax.hist(y_samples, bins=20, alpha=0.3, facecolor='b', edgecolor='k')
ax.hist(x_samples, bins=20, alpha=0.5, facecolor='r', edgecolor='k')

plt.show()

# AMISC tutorial (part 2)

## Review

Multidisciplinary (MD) systems are constructed from 3 objects:

- **Variables** - These are inputs, outputs, QoIs, etc. They are the most basic element of any MD system and serve as the datapaths or connections between models. They can be random variables, scalars, field quantities, etc.
- **Components** - These are the individual discipline models. They map a set of inputs to a set of outputs.
- **System** - This is the top-level MD system. It connects multiple component models and manages the flow of data between them

In [ ]:
from amisc import Variable, Component, System

### Variables

In [ ]:
x = Variable()                                                 # Empty variable
inputs = [Variable(f'x{i}') for i in range(3)]                 # Giving names
theta = Variable(name='t',                                     # Random variable
                 tex=r'$\theta$', 
                 units='rad', 
                 description='Fit coefficient', 
                 category='calibration', 
                 distribution='N(0, 1)'
                )
pressure = Variable(name='p',                                  # Field quantity
                    shape=(100, 3), 
                    compression=dict(method='svd', rank=2)
                   )

In [ ]:
print(x)
print(inputs)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(theta.sample(1000), bins=20, alpha=0.5, facecolor='r', edgecolor='k')
plt.xlabel(theta.get_tex(units=True))
plt.show()

In [ ]:
# Loading from file
config = """
!Variable
name: t
tex: '$\\theta$'
units: K
description: Fit coefficient
distribution: N(0, 1)
"""

with open('file.yml', 'w') as fd:
    fd.write(config)

In [ ]:
from amisc import YamlLoader
theta = YamlLoader.load('file.yml')

In [ ]:
plt.hist(theta.sample(1000), bins=20, alpha=0.5, facecolor='r', edgecolor='k')
plt.xlabel(theta.get_tex(units=True))
plt.show()

### Components

\begin{align}
y_1 &= f_1(x) = x \sin(\pi x)\\
y_2 &= f_2(y_1) = \frac{1}{1 + 25y_1^2}\\
y_3 &= f_3(x, y_2) = x \cos(\pi y_2)
\end{align}

In [ ]:
# Simple components
def f1(x):
    y1 = x * np.sin(np.pi * x)
    return y1
def f2(y1):
    y2 = 1 / (1 + 25*y1**2)
    return y2
comp1 = Component(f1)
comp2 = Component(f2)
print(comp1)
print(comp2)

In [ ]:
# Multiple inputs
def f3(x, y2):
    y3 = x * np.cos(np.pi * y2)
    return y3

# Alternatively
def f3_dict(inputs):
    return { 'y3': inputs['x'] * np.cos(np.pi * inputs['y2']) }

comp3 = Component(f3)
comp3_dict = Component(f3_dict, inputs=[Variable('x'), Variable('y2')], outputs=[Variable('y3')])
print(comp3)
print(comp3_dict)

In [ ]:
# Loading from file
config = """
!Component
name: My component 3
model: !!python/name:amisc.tutorial.f3_dict
inputs:
  !Variable
  - name: x
  - name: y2
outputs:
  !Variable
  - name: y3
"""

with open('file.yml', 'w') as fd:
    fd.write(config)

In [ ]:
comp3 = YamlLoader.load('file.yml')
print(comp3)

In [ ]:
comp3.predict({'x': 2.0, 'y2': 1.0}, use_model='best')

### System

In [ ]:
from amisc import YamlLoader
sys = System(f1, f2, f3)
print(sys)

In [ ]:
# The only two functions you need to get going
# sys.fit()
# sys.predict()

In [ ]:
# Loading from file
config = """
!System
name: My multidisciplinary system
components:
  !Component
  - model: !!python/name:amisc.tutorial.f1
  - model: !!python/name:amisc.tutorial.f2
  - model: !!python/name:amisc.tutorial.f3
"""

with open('file.yml', 'w') as fd:
    fd.write(config)

In [ ]:
from amisc import YamlLoader
sys = YamlLoader.load('file.yml')
print(sys)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x_grid = {'x': np.linspace(0, 1, 100)}

In [ ]:
y_grid = sys.predict(x_grid, use_model='best')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), layout='tight', sharey='row')
ax[0].plot(x_grid['x'], y_grid['y1'], '-r')
ax[1].plot(x_grid['x'], y_grid['y2'], '-r')
ax[2].plot(x_grid['x'], y_grid['y3'], '-r')
ax[0].set_xlabel('$x$')
ax[1].set_xlabel('$x$')
ax[2].set_xlabel('$x$')
ax[0].set_ylabel('$f(x)$')
plt.show()

## Feedback cycles

Feedback cycles are handled by converting the multidisciplinary system to a directed acyclic graph (DAG) and computing cycles with a nonlinear fixed-point iteration (FPI) routine.

For example:

\begin{align}
y_1 &= f_1(x, y_2) = -x^3 + 2y_2^2\\
y_2 &= f_2(x, y_1) = 3x^2 + 4y_1^{-2}\\
\end{align}

To solve with FPI:

- Choose an initial sample for the coupling variables: $\xi^{(0)} = (y_1^{(0)}, y_2^{(0)})$
- Iterate $\xi^{(i+1)} = f(\xi^{(i)})$ until $|\xi^{(i+1)}-\xi^{(i)}| < \epsilon$

This is essentially a solver for a nonlinear system of equations (iterate until a residual is below a tolerance, i.e. Newton-Raphson)

In [ ]:
f1 = lambda inputs: {'y1': -inputs['x']**3 + 2 * inputs['y2']**2}
f2 = lambda inputs: {'y2': 3*inputs['x']**2 + 4 * inputs['y1']**(-2)}
x = Variable(distribution='U(0, 4)')
y1 = Variable(distribution='U(1, 10)')
y2 = Variable(distribution='U(1, 10)')
comp1 = Component(f1, [x, y2], y1)
comp2 = Component(f2, [x, y1], y2)
surr = System(comp1, comp2)

In [ ]:
import networkx as nx
nx.draw(surr.graph, with_labels=True, font_weight='bold')

In [ ]:
tol = 1e-10
# inputs = surr.sample_inputs(100)
inputs = {'x': 0}  # (y1=2, y2=1) analytic solution
y_surr = surr.predict(inputs, use_model='best', max_fpi_iter=200, fpi_tol=tol)

## File configuration

The three main `amisc` objects can be defined directly from yaml config files. Very similar to json.

In [ ]:
config = 
"""
!System
components: !Component
  - name: comp1
    model: ...
    inputs: !Variable
    - name: x1
    - name: x2
    outputs: !Variable
    - name: y1
    - name: y2
  - name: comp2
    model: ...
    ...
"""

surr = System.load_from_file('surr.yml')
surr.save_to_file('surr-copy.yml')

## Putting it all together

In [ ]:
from amisc.examples.models import fire_sat_system

fire_sat = fire_sat_system()
print(fire_sat)

In [ ]:
import networkx as nx
nx.draw(fire_sat.graph, with_labels=True, font_weight='bold')

In [ ]:
fire_sat.set_logger(stdout=True)

inputs = fire_sat.sample_inputs(1000)
outputs = fire_sat.predict(inputs, use_model='best', verbose=True, fpi_tol=1e-12)

In [ ]:
# Plot error histograms
import matplotlib.pyplot as plt

fig_in, ax_in = plt.subplots(1, 3, figsize=(9, 3), layout='tight')
fig_out, ax_out = plt.subplots(1, 3, figsize=(9, 3), layout='tight')
for i in range(3):
    input_var = fire_sat.inputs()[i]
    output_var = fire_sat.outputs()[i]
    ax_in[i].hist(inputs[input_var], color='blue', bins=20, edgecolor='black', density=True, linewidth=1.2)
    ax_out[i].hist(outputs[output_var], color='red', bins=20, edgecolor='black', density=True, linewidth=1.2)

    ax_in[i].set_xlabel(input_var.get_tex())
    ax_out[i].set_xlabel(output_var.get_tex())

plt.show()

## Advanced features

### Parallelization

You can compute the model/surrogate in 3 ways:

- Serial - by default
- Vectorized - if your component models allow it
- Parallel - by passing in a parallel executor

The call signature is identical no matter the method or whether you are using the surrogate or underlying models.

In [ ]:
from concurrent.futures import ThreadPoolExecutor

x_grid = {'x': np.linspace(0, 1, 100)}

y_grid_serial = system.predict(x_grid)
y_grid_vector = system.predict(x_grid, vectorized=True)
y_grid_parallel = system.predict(x_grid, executor=ThreadPoolExecutor(max_workers=4))

### Field quantities (high-dimensional)

In [ ]:
from amisc.utils import relative_error

N_grid = 200
amp = Variable('amp', distribution='U(-5, 5)')
pressure = Variable('p', shape=(N_grid,), compression=dict(method='svd', energy_tol=0.99))
def model(inputs, coords):
    amp = inputs['amp']
    return {'p': amp[..., np.newaxis] * np.tanh(np.squeeze(coords))}

# Generate SVD data matrix
params = amp.sample(50)
x_grid = np.linspace(-2, 2, N_grid)
outputs = model({str(amp): params}, x_grid)
data_matrix = outputs[pressure].T  # (dof, num_samples)

pressure.compression.coords = x_grid
pressure.compression.compute_map(data_matrix)

# Test compression
coarse_shape = 25
coarse_grid = np.linspace(-2, 2, coarse_shape)
num_test = (5, 20)
params = amp.sample(num_test)
outputs = model({str(amp): params}, coarse_grid)

outputs_latent = pressure.compress(outputs, coords=coarse_grid)
outputs_reconstruct = pressure.reconstruct(outputs_latent)

y = outputs[pressure]
yhat = outputs_reconstruct[pressure]

assert relative_error(yhat, y) < 0.01

### Complex models

Other things you can do in your models:

- Request an `output_path` for saving simulation output/result files
- Make arbitrary calls to other programs via `subprocess.run('my_program.exe')`

In [ ]:
def my_model(inputs, output_path='.'):
    # Compute your model here
    # ...
    # subprocess.run('my_model.exe')

    # Save outputs to file
    from pathlib import Path
    data = np.random.rand(100, 10)
    with open(Path(output_path) / 'my_file.dat', 'w') as fd:
        np.savetxt(fd, data)

### `amisc` customization

- Use Gaussian Process regression instead of Lagrange polynomials
- Use alternative compression methods
- Use alternative serialization methods
- Use alternative data storage methods

In [ ]:
from amisc.interpolator import Interpolator  # Interpolation interface  (i.e. Lagrange)
from amisc.compression import Compression    # Compression interface    (i.e. SVD)
from amisc.serialize import Serializable     # Serialization interface
from amisc.training import TrainingData      # Data storage interface   (i.e. SparseGrid)

In [ ]:
# For example
class MyCompression(Compression):
    def compress():
        pass
    def reconstruct():
        pass